**Phase 5: Fine-Tuning Using LoRA or QLoRA**

In [2]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training
)
from datasets import Dataset
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import torch

In [12]:
from huggingface_hub import login
# Login to Hugging Face Hub
login()

In [4]:
model_name = "meta-llama/Llama-3.2-1B"
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [5]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Toggle LoRA vs QLoRA
use_qlora = True  # set False to use standard LoRA (full precision)

if use_qlora:
    # QLoRA: 4-bit quantization via Transformers' BitsAndBytesConfig
    quant_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        quantization_config=quant_config,
        trust_remote_code=True,
    )
    model = prepare_model_for_kbit_training(model)
else:
    # Standard LoRA: full-precision adapter
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16,
        trust_remote_code=True,
    )


In [6]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # adjust based on model internals
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

In [7]:
embed_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma(
    collection_name="database",
    embedding_function=embed_model,
    persist_directory="database"
)
texts = vectorstore.get().get("documents", [])  # list[str]
print(f"Number of documents: {len(texts)}")
hf_dataset = Dataset.from_dict({"text": texts})

C:\Users\krish\AppData\Local\Temp\ipykernel_7628\3420278840.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
C:\Users\krish\AppData\Local\Temp\ipykernel_7628\3420278840.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


Number of documents: 1731


In [8]:
def tokenize_fn(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

# Tokenize & batch
tokenized_ds = hf_dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=["text"]
)

Map:   0%|          | 0/1731 [00:00<?, ? examples/s]

In [9]:
training_args = TrainingArguments(
    output_dir="./phase5_lora_llama3.2-1b",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    warmup_steps=100,
    num_train_epochs=3,                   
    learning_rate=2e-4,
    fp16=True,
    logging_steps=20,
    save_steps=200,
    optim="paged_adamw_8bit",
    remove_unused_columns=False,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [10]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_ds,
    args=training_args,
    data_collator=data_collator,
)

trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: krishshandilya18 (krishshandilya18-pes-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\_dynamo\eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
20,2.801100
40,2.697800
60,2.582500
80,2.496900
100,2.447700
120,2.363000
140,2.364600
160,2.341000


c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\peft\utils\other.py:1110: UserWarning: Unable to fetch remote file due to the following error (Request ID: Root=1-6807876d-4a0987c475235870241f7000;ccea70cc-9b1a-4d6e-8063-98f0431db587)

403 Forbidden: Please enable access to public gated repositories in your fine-grained token settings to view this repository..
Cannot access content at: https://huggingface.co/meta-llama/Llama-3.2-1B/resolve/main/config.json.
Make sure your token has the correct permissions. - silently ignoring the lookup for the file config.json in meta-llama/Llama-3.2-1B.
  warnings.warn(
c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\peft\utils\save_and_load.py:236: UserWarning: Could not find a config file in meta-llama/Llama-3.2-1B - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=162, training_loss=2.510652580378968, metrics={'train_runtime': 1655.4421, 'train_samples_per_second': 3.137, 'train_steps_per_second': 0.098, 'total_flos': 1.533761789362176e+16, 'train_loss': 2.510652580378968, 'epoch': 2.9607390300230945})

In [13]:
model.save_pretrained("./phase5_lora_llama3.2-1b")
tokenizer.save_pretrained("./phase5_lora_llama3.2-1b")

('./phase5_lora_llama3.2-1b\\tokenizer_config.json',
 './phase5_lora_llama3.2-1b\\special_tokens_map.json',
 './phase5_lora_llama3.2-1b\\tokenizer.json')

In [14]:
from huggingface_hub import HfApi

repo_name = "CHESSBOT_GENAI_llama3.2-1b"

model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

api = HfApi()
print(f"Model uploaded to: https://huggingface.co/{api.whoami()['name']}/{repo_name}")

adapter_model.safetensors:   0%|          | 0.00/3.42M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

c:\Users\krish\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\krish\.cache\huggingface\hub\models--PES1UG22CS292--CHESSBOT_GENAI_llama3.2-1b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Model uploaded to: https://huggingface.co/PES1UG22CS292/CHESSBOT_GENAI_llama3.2-1b
